### Imports and General Data

In [14]:
import pandas as pd
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sqlite3
import pandas as pd
from preprocessing import create_connection, get_files
from main import get_files, get_data, export_data
(root, data_path, presidents, cities, countries, years, colors) = get_data()

# Data Overview

In [ ]:
trump = pd.read_pickle(f'{data_path}trump.pkl')
johnson = pd.read_pickle(f'{data_path}johnson.pkl')

In [ ]:
def get_overview(filepath):
    data = pd.read_pickle(filepath)
    shape = data.shape
    cols = data.columns
    print(f'Shape: {shape}, Cols: {cols}')
    return shape, cols

In [ ]:
trump.shape, johnson.shape

((0, 0), (0, 0))

In [15]:
trump.describe

<bound method NDFrame.describe of         item_number                   id                      created_at  \
0               136  1024792112146071552  Wed Aug 01 23:00:53 +0000 2018   
1               687  1024768368077930496  Wed Aug 01 21:26:32 +0000 2018   
2              1988  1024727636285812736  Wed Aug 01 18:44:41 +0000 2018   
3              2069  1024725157796098048  Wed Aug 01 18:34:50 +0000 2018   
4              2233  1024720439912873992  Wed Aug 01 18:16:05 +0000 2018   
...             ...                  ...                             ...   
1634984      772159  1542300679372513281  Thu Jun 30 00:15:11 +0000 2022   
1634985      772256  1542299493227446278  Thu Jun 30 00:10:28 +0000 2022   
1634986      772286  1542299155997007877  Thu Jun 30 00:09:07 +0000 2022   
1634987      772344  1542298383217459202  Thu Jun 30 00:06:03 +0000 2022   
1634988      772434  1542297372180578304  Thu Jun 30 00:02:02 +0000 2022   

        created_timestamp                            

In [16]:
trump.columns

Index(['item_number', 'id', 'created_at', 'created_timestamp', 'text',
       'quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'lang',
       ...
       'nonflu', 'filler', 'AllPunc', 'Period', 'Comma', 'QMark', 'Exclam',
       'Apostro', 'OtherP', 'Emoji'],
      dtype='object', length=136)

In [3]:
trump = pd.read_pickle(f'{data_path}trump.csv.pkl')

In [5]:
trump.shape

(1634989, 136)

In [13]:
trump['lang'].unique()

array(['en'], dtype=object)

In [8]:
trump[trump['lang'] in ['GB', '']]

Index(['item_number', 'id', 'created_at', 'created_timestamp', 'text',
       'quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'lang',
       'geo_lat', 'geo_lon', 'place_id', 'Compound', 'Positive', 'Negative',
       'Neutral', 'Segment', 'WC', 'Analytic'],
      dtype='object')

In [ ]:
trump.drop(columns=[])

In [ ]:
# trump[['created_at', 'created_timestamp']]
trump[['created_timestamp']]
# format: UTC time ()

### Location Analysis

Average sentiment score in each city and country

In [ ]:
# TODO: run sql query to find out average
# get number of rows in sentiment column
# get average of 'Compound'

In [ ]:
conn = create_connection(db_file)
cur = conn.cursor()
cur.execute(f'SELECT {select} FROM {table} WHERE {where} LIMIT {limit}')

rows = cur.fetchall()

for row in rows:
    print(row)


In [17]:
def get_sum_and_count(db_file, table_name, col):
    """Get the sum and count of the 'Compound' column from the specified table."""
    conn = create_connection(db_file)
    if conn:
        query = f"""SELECT SUM({col}) as sum_col, COUNT(*) as count FROM {table_name}"""
        result = pd.read_sql_query(query, conn)
        sum_col = result.iloc[0]['sum_col']
        count = result.iloc[0]['count']
        conn.close()
        return sum_col, count
    else:
        return None

In [20]:
for city in cities:
    for year in years:
        for db_file in get_files(f'{root}{city}/{year}/'):
            compound_sum, compound_num_of_rows = get_sum_and_count(f'{root}{city}/{year}/{db_file}', 'sentiment', 'Compound')

KeyboardInterrupt: 

In [21]:
get_files(f'{root}{city}/')

['2018', '2019', '2020', '2021', '2022']

In [ ]:
# List of database files
db_files = ['path/to/db1.db', 'path/to/db2.db', ...]  # Add your database file paths here

# Calculate the sum and count of Compound values for each database
sums_counts = [get_sum_and_count(db_file) for db_file in db_files if get_sum_and_count(db_file) is not None]

# Calculate the overall sum and count
overall_sum = sum([item[0] for item in sums_counts])
overall_count = sum([item[1] for item in sums_counts])

# Calculate the overall average if there are valid sums and counts
if overall_count > 0:
    overall_average = overall_sum / overall_count
    print("Overall average of Compound:", overall_average)
else:
    print("No valid data to calculate average.")

Average sentiment of the presidents in each city and country

In [ ]:
for city in cities:
    trump = pd.read_pickle(f'{data_path}{presidents[0]}-{city}.pkl')
    johnson = pd.read_pickle(f'{data_path}{presidents[1]}-{city}.pkl')

In [2]:
trump = pd.read_pickle(f'{data_path}{presidents[0]}-Birmingham.pkl')

FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/Festplatte/data-UA/data/trump-Birmingham.pkl'

In [10]:
trump.shape

(26758, 136)

In [11]:
trump.columns

Index(['item_number', 'id', 'created_at', 'created_timestamp', 'text',
       'quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'lang',
       ...
       'nonflu', 'filler', 'AllPunc', 'Period', 'Comma', 'QMark', 'Exclam',
       'Apostro', 'OtherP', 'Emoji'],
      dtype='object', length=136)

##### Birmingham

In [ ]:
trump = pd.read_pickle(f'{data_path}trump.pkl')
johnson = pd.read_pickle(f'{data_path}johnson.pkl')

##### LA